In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer   #for bag of words
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup   #to clean data (remove html stuff)
import re   #regular expression, used to remove '/' fromm data
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords   #to clean stop words

Aim: to first clean html tags and stop words. then, create bag of words. then, convert text based datd into numbers and matrix called bag of words- done by count vectorizer

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving labeledTrainData.tsv.zip to labeledTrainData.tsv.zip


In [ ]:
!unzip labeledTrainData.tsv.zip
df=pd.read_csv('labeledTrainData.tsv', delimiter="\t", quoting=3)    #delimteter acts as seperator instead of ',' in csv files, quoting=3 means quoting=QUOTE_NONE which means remove string quotes if found any
                                                                      #but puts " "" " quotes in this case dk how, prolly in dataset but removed here in DF
df.head()



Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
len(df)

25000

WHAT ARE **STOPWORDS**: stopwords are words like 'the', 'a', 'of','it','at','with' etc

In [ ]:
nltk.download('stopwords')    #NLP do not use stopwords hence we have to remove them

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

DATA CLEANING: 

In [ ]:
#removing html tags and letters like "/", <br> etc
#doing it step by step with first row for understanding:
sample=df.review[0]
sample

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [ ]:
#clear html tags:
sample=BeautifulSoup(sample).get_text() #remove html tags
sample
#but '/' are still there  

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [ ]:
#remove '/':
sample=re.sub("[^a-zA-Z]", ' ', sample)
sample

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [ ]:
#converting to lowercase beacause ML thinks capital letters w diff words:
sample=sample.lower()
sample

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [ ]:
#to remove stopwords-grammar words:
sample=sample.split()
stopwordsvar=set(stopwords.words("english"))    #saves all stopwords in variable
sample=[w for w in sample if w not in stopwordsvar]
sample
#crosscheck using lengths

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [ ]:
#making a func for above:
def cleaning(review):
  #remove html tags
  review=BeautifulSoup(review).get_text()
  #remove '/'
  review=re.sub("[^a-zA-Z]", ' ', review)
  review=review.lower()
  review=review.split()
  #remove stop words
  swords=set(stopwords.words("english"))
  review=[w for w in review if w not in swords]

  return (" ".join(review))


In [ ]:
train_x_tum=[]
for r in range(len(df["review"])):
  if (r+1)%1000==0:
    print("no of processed review= ",r+1)
  train_x_tum.append(cleaning(df['review'][r]))

no of processed review=  1000
no of processed review=  2000
no of processed review=  3000
no of processed review=  4000
no of processed review=  5000
no of processed review=  6000
no of processed review=  7000
no of processed review=  8000
no of processed review=  9000
no of processed review=  10000
no of processed review=  11000
no of processed review=  12000
no of processed review=  13000
no of processed review=  14000
no of processed review=  15000
no of processed review=  16000
no of processed review=  17000
no of processed review=  18000
no of processed review=  19000
no of processed review=  20000
no of processed review=  21000
no of processed review=  22000
no of processed review=  23000
no of processed review=  24000
no of processed review=  25000


In [ ]:
#train test split:
x=train_x_tum
y=np.array(df["sentiment"])

#split:
train_x, test_x, y_train, y_test=train_test_split(x,y, test_size=0.1)

BAG OF WORDS: converting texts data into numbers:

In [ ]:
vectorizer= CountVectorizer(max_features=5000)    #max words in bag=5000, trail and error to get better acc

#convert train data to feature vector matrix:
train_x=vectorizer.fit_transform(train_x)

#this will make a matrix w all the words as col and review no as row and will put the occurenece of the word as number in blocks
#did this as one hot coding in AI ML DL course


In [ ]:
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1779849 stored elements in Compressed Sparse Row format>

In [ ]:
#to array:
train_xx=train_x.toarray()
train_y=y_train

train_xx.shape, train_y.shape

((22500, 5000), (22500,))

In [ ]:
train_y


array([1, 1, 1, ..., 0, 0, 0])

random forest

In [ ]:
model= RandomForestClassifier(n_estimators=150, random_state=42)  #100 decision trees, to get same results everytime: 42, change decision tree values for better acc
model.fit(train_x, train_y)

RandomForestClassifier(n_estimators=150, random_state=42)

Testing

In [ ]:
#convert testing data to feature vector matrix:
test_xx= vectorizer.transform(test_x)

test_xx

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 195377 stored elements in Compressed Sparse Row format>

In [ ]:
test_xx=test_xx.toarray()
test_xx.shape

(2500, 5000)

In [ ]:
#one hot coding: to vectorize:
#def vectorize_sequences(sequences, dimension=5000):
#    results=np.zeros((len(sequences), dimension))
#    for i, sequence in enumerate(sequences):
#        results[i, sequence]=1
#    return results

In [ ]:
#x_train=vectorize_sequences(test_x)
#x_test=vectorize_sequences(test_y)

#y_train=np.asarray(test_x).astype('float32')
#y_test=np.asarray(test_y).astype('float32')

In [ ]:
#Predict:
test_predict=model.predict(test_xx)
acc=roc_auc_score(y_test, test_predict)
acc

0.8500239052392475